In [2]:
#Libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#Load the datasets

#This function retrieves the list of text files (which are all indivudial arguments) and compiles them into a dataframe
def get_arguments_from_text():
    path = "/Users/danielbradley/Documents/Github/FlatIron/Phase5/Sophie-Debate-Bot/data/no_wavs/trs.txt"
    os.chdir(path)
    list_of_files = []

    # iterate through all file
    for file in os.listdir():
        list_of_files.append(file)

    #Create a list of arguments as strings
    list_of_strings = []
    for file in list_of_files:
        with open(path+'/'+file) as f:
            list_of_strings.append(f.read())

    #Turn the list into a dataframe
    return pd.DataFrame(list_of_strings, columns=['argument'])

#1: Argument detection dataset
dataset_arg_detect = pd.read_csv('data/IBM_Debater_(R)_ArgsInASR_Findings-2020.v1/argumentative_sentences_in_spoken_language_with split.csv')
dataset_arg_detect_train = pd.read_csv('data/IBM_Debater_(R)_ArgsInASR_Findings-2020.v1/data_sets/q_mc_train.csv')
dataset_arg_detect_test = pd.read_csv('data/IBM_Debater_(R)_ArgsInASR_Findings-2020.v1/data_sets/q_mc_test.csv')
dataset_arg_detect_wiki = pd.read_csv('data/IBM_Debater_(R)_ArgsInASR_Findings-2020.v1/wikipedia_evidence_dataset_29429.csv')

#2: Data quality dataset - test and train
dataset_arg_qual_train = pd.read_csv('data/IBM_Debater_(R)_EviConv-ACL-2019.v1/train.csv')
dataset_arg_qual_test = pd.read_csv('data/IBM_Debater_(R)_EviConv-ACL-2019.v1/test.csv')

#3: Argument stance detection dataset
dataset_arg_stance = pd.read_csv('data/IBM_Debater_(R)_CS_EACL-2017.v1/claim_stance_dataset_v1.csv')

#4: Debater analysis dataset
dataset_arg_gen = pd.read_csv('data/no_wavs/RecordedDebatingDataset_Release5_metadata.csv')
dataset_arg_gen_ootec = pd.read_csv('data/no_wavs/OutOfTheEchoChamber_acl2020_split.csv')

owd = os.getcwd()
dataset_arg_gen_args = get_arguments_from_text()
os.chdir(owd)

#5: Key point analysis: Arg KP dataset
dataset_key_point = pd.read_csv('data/IBM_Debater_(R)_ArgKP-2021/ArgKP-2021_dataset.csv')

#6: Claim generation: Must be able to generate claims
dataset_claim_gen = pd.read_csv('data/IBM_Debater_(R)_ GPT-2_Claim_Generation/generated_texts_plausibility_stance.csv')

## Dataset #1: Domain detection
What topic is the debater discussing? The model must be able to determine this accurately to access the correct class of responses.

- a. The dataset: motion-evidence samples taken from Wikipedia with acceptance rate scored from 0-1 (acceptance rate being confidence that the text contains evidence for the motion)
- b. over 29,000 evidence claims with relevant motion and concept

Goals of the model: Should be able to determine which topic is being discussed and whether the sentence contains evidence on the motion or not

In [4]:
#Generating the relevant target and feature columns. The data  column is both the motion and evidence, as this helps the model train on a GIVEN topic (see Liat Ein-Dor et al., 2020)

dataset_arg_detect_wiki['target'] = dataset_arg_detect_wiki['acceptanceRate'].apply(lambda x: 1 if x > 0.5 else 0)
dataset_arg_detect_wiki = dataset_arg_detect_wiki.drop(columns= ['Wikipedia article_id', 'Wikipedia article_title'])
dataset_arg_detect_wiki['data'] = dataset_arg_detect_wiki['Evidence'] + dataset_arg_detect_wiki['Motion Text']

In [5]:
dataset_arg_detect_wiki.sort_values(by = 'acceptanceRate', ascending=True).loc[dataset_arg_detect_wiki['Dominant Concept'] == 'Doping in sport'][:]

,Motion Text,Dominant Concept,Evidence,acceptanceRate,target,data
145,We should legalize doping in sport,Doping in sport,A routine unannounced drug test found the bann...,0.0,0,A routine unannounced drug test found the bann...
154,We should legalize doping in sport,Doping in sport,A report by a joint International Swimming Fed...,0.0,0,A report by a joint International Swimming Fed...
155,We should legalize doping in sport,Doping in sport,"In his book Clearing the Bases, he said that a...",0.0,0,"In his book Clearing the Bases, he said that a..."
160,We should legalize doping in sport,Doping in sport,Matthew Syed countered that while the incident...,0.0,0,Matthew Syed countered that while the incident...
166,We should legalize doping in sport,Doping in sport,After Landis' A sample tested positive for tes...,0.0,0,After Landis' A sample tested positive for tes...
...,...,...,...,...,...,...
77,We should legalize doping in sport,Doping in sport,"It is estimated that around 10,000 former athl...",1.0,1,"It is estimated that around 10,000 former athl..."
152,We should legalize doping in sport,Doping in sport,Another point of view is that doping could be ...,1.0,1,Another point of view is that doping could be ...
80,We should legalize doping in sport,Doping in sport,A recent study has also shown that long term A...,1.0,1,A recent study has also shown that long term A...
112,We should legalize doping in sport,Doping in sport,"A 2005 review in CNS Drugs determined that ""si...",1.0,1,"A 2005 review in CNS Drugs determined that ""si..."


In [8]:
dataset_arg_detect_wiki.to_csv('processed_data/dataset_arg_detect_wiki.csv')


## Model training is done on Google Colab. Re-import here when it's complete.